In [14]:
%load_ext bigdata

The bigdata extension is already loaded. To reload it, use:
  %reload_ext bigdata


In [15]:
%hive_start

In [16]:
%timeout 300

In [17]:
!hadoop fs -rm -f -r tmp;

In [18]:
## Borra la carpeta si existe
!hdfs dfs -rm -r -f /tmp/output
## Crea la carpeta drivers en el HDFS
!hdfs dfs -mkdir /tmp/output
## Copia los archivos al HDFS
!hdfs dfs -copyFromLocal data.tsv  /tmp/output/data.tsv
## Lista los archivos al HDFS para verificar
## que los archivos fueron copiados correctamente.
!hdfs dfs -ls /tmp/output/*

Deleted /tmp/output
-rw-r--r--   1 root supergroup        637 2020-01-20 00:49 /tmp/output/data.tsv


In [19]:
%%hive

DROP TABLE IF EXISTS data;

CREATE TABLE data (letra       STRING,
                           fecha        STRING,
                           numero      INT)

ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE data;

DROP TABLE IF EXISTS data;
FAILED: SemanticException org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient
CREATE TABLE data (letra       STRING,
                           fecha        STRING,
                           numero      INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';
FAILED: SemanticException org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE data;
FAILED: SemanticException org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient


In [20]:
%%hive
--guardamos la consulta en una tabla
DROP TABLE IF EXISTS salida;
CREATE TABLE salida
AS
    SELECT concat(letra,
        fecha, 
        numero) from docs;
    

--guardamos la consulta en una tabla
DROP TABLE IF EXISTS salida;
FAILED: SemanticException org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient
CREATE TABLE salida
AS
    SELECT concat(letra,
        fecha, 
        numero) from docs;
FAILED: SemanticException org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient


In [ ]:
%%hive
DROP TABLE IF EXISTS docs;

In [ ]:
%%hive
INSERT OVERWRITE DIRECTORY 'tmp/000000_0'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
Select * from salida;

select concat(letra,',', count(*)) as junto from data
group by letra
order by junto;

In [ ]:
!hadoop fs - copyToLocal ./tmp/000000_0 output 

In [ ]:
%hive_quit;

In [114]:
!hive -S -e 'source wordcount.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
FAILED: SemanticException org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient


In [ ]:
## Se lista el contenido del archivo.
!hdfs dfs -ls /tmp/output

In [ ]:
## se visualiza la cabecera del archivo.
!hdfs dfs -cat /tmp/output/000000_0 | head

In [112]:
##Copia de los resultados a la maquina local
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

In [ ]:
!cat output/000000_0

%%hive
DROP TABLE IF EXISTS docs;

CREATE TABLE docs (letra       STRING,
                           fecha        STRING,
                           numero      INT)

ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';



In [103]:
%%hive
LOAD DATA INPATH "/tmp/wordcount/" OVERWRITE INTO TABLE docs;
select concat(letra,',', count(*)) as junto from docs
group by letra
order by junto;



LOAD DATA LOCAL INPATH "wordcount/" OVERWRITE INTO TABLE docs;
Loading data to table default.docs
OK
Time taken: 2.162 seconds
select concat(letra,',', count(*)) as junto from docs
group by letra
order by junto;
Query ID = root_20200119230950_e21f0cd8-6e65-46ff-a180-07cf69873186
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579467843452_0012, Tracking URL = http://0986141801dd:8088/proxy/application_1579467843452_0012/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579467843452_0012
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-01-19 23:10:09,856 Stage-1 map = 0%,  redu

In [104]:
##
## Se usan un directorio temporal en el HDFS. La siguiente
## instrucción muestra el contenido del dicho directorio
##
!hdfs dfs -ls /tmp

Found 4 items
drwxr-xr-x   - root supergroup          0 2020-01-19 22:24 /tmp/data
drwxrwx---   - root supergroup          0 2020-01-19 21:05 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-01-19 21:09 /tmp/hive
drwxr-xr-x   - root supergroup          0 2020-01-19 23:09 /tmp/output


In [105]:
##
## Crea la carpeta wordcount en el hdfs
##
!hdfs dfs -mkdir /tmp/wordcount

In [106]:
##
## Verifica la creación de la carpeta
##
!hdfs dfs -ls /tmp/

Found 5 items
drwxr-xr-x   - root supergroup          0 2020-01-19 22:24 /tmp/data
drwxrwx---   - root supergroup          0 2020-01-19 21:05 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-01-19 21:09 /tmp/hive
drwxr-xr-x   - root supergroup          0 2020-01-19 23:09 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-01-19 23:14 /tmp/wordcount


In [107]:
##
## Copia los archvios del directorio local wordcount/
## al directorio /tmp/wordcount/ en el hdfs
##
!hdfs dfs -copyFromLocal wordcount/*  /tmp/wordcount/

In [108]:
##
## Verifica que los archivos esten copiados
## en el hdfs
##
!hdfs dfs -ls /tmp/wordcount

Found 1 items
-rw-r--r--   1 root supergroup        637 2020-01-19 23:15 /tmp/wordcount/data.tsv


In [111]:
LOAD DATA LOCAL INPATH "wordcount/" OVERWRITE INTO TABLE docs;

SyntaxError: invalid syntax (<ipython-input-111-4ebaa27ddd88>, line 1)

In [80]:
%%hive
SHOW TABLES;
SHOW CREATE TABLE data;

SHOW TABLES;
OK
data
docs
prueba
Time taken: 0.228 seconds, Fetched: 3 row(s)
SHOW CREATE TABLE data;
OK
CREATE TABLE `data`(
  `letra` string, 
  `fecha` string, 
  `numero` int)
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe' 
WITH SERDEPROPERTIES ( 
  'field.delim'='\t', 
  'serialization.format'='\t') 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  'hdfs://0.0.0.0:9000/user/hive/warehouse/data'
TBLPROPERTIES (
  'transient_lastDdlTime'='1579472177')
Time taken: 0.155 seconds, Fetched: 17 row(s)


In [81]:
%%hive
DESCRIBE data;

DESCRIBE data;
OK
letra               	string              	                    
fecha               	string              	                    
numero              	int                 	                    
Time taken: 0.093 seconds, Fetched: 3 row(s)


In [82]:
%%hive
SELECT * FROM data LIMIT 5;

SELECT * FROM data LIMIT 5;
OK
B	1999-08-28	14
E	1999-12-06	12
E	1993-07-21	17
C	1991-02-12	13
E	1995-04-25	16
Time taken: 2.715 seconds, Fetched: 5 row(s)


In [85]:
%%hive
select concat(letra,',', count(*)) as junto from data
group by letra
order by junto;

select concat(letra,',', count(*)) as junto from data
group by letra
order by junto;
Query ID = root_20200119224734_c7fdab67-411c-4a9d-84be-d9f5b5f99bec
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579467843452_0008, Tracking URL = http://0986141801dd:8088/proxy/application_1579467843452_0008/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579467843452_0008
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-01-19 22:47:49,540 Stage-1 map = 0%,  reduce = 0%
2020-01-19 22:47:58,119 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 3.4 sec
2020-01-19 22:48:07,508 Stage-1 map = 

%hive_quit